<h1 style="background-color:red;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת שכבת חלקות (דו מימדיות) מבוטלות לבנק"ל מודרני 🌐

In [ ]:
print('15. Canceled parcels borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  15.a Reading inputs')

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today =  dt.date.today()
product_name =  str(today.year) + str(today.month) + str(today.day)  + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name
del [today, product_name]

<h1 style="text-align:right;font-size:125%">
נתוני שכבת חלקות היסטוריות

In [ ]:
history_parcels_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'PARCEL_TYPE', 'LEGAL_AREA', 'TALAR_NUM', 'TALAR_YEAR', 'geometry']

history_parcels = gpd.read_file(RawData_Folder + 'CADSDE\PARCEL_HISTORY_HASAVA.gdb',
                                layer = 'PARCEL_HISTORY_HASAVA',
                                include_fields = history_parcels_cols)
del history_parcels_cols

print('         input 1/7 read')

<h1 style="text-align:right;font-size:125%">
טבלת חלקות מבוטלות

In [ ]:
cancel_parcel_cols = ['F_GUSH_NUM', 'F_GUSH_SUFFIX', 'F_PARCEL_NUM', 'TALAR_NUMBER', 'TALAR_YEAR']

cancel_parcel = pd.read_csv(RawData_Folder + 'Oracle\CANCEL_PARCEL.csv', usecols = cancel_parcel_cols)
del cancel_parcel_cols

print('         input 2/7 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות מבוטלות מרפליקציות יומיות

In [ ]:
replicated_parcels_cols = ['PARCEL_ID', 'PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGALAREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

replicated_parcels = gpd.read_file(RawData_Folder + 'Local\CanceldParcels.gdb',
                                   layer = 'CanceledParcels',
                                   include_fields = replicated_parcels_cols)
del replicated_parcels_cols


print('         input 3/7 read')

<h1 style="text-align:right;font-size:125%">
נתוני גושים מודרניים 

In [ ]:
blocks_cols = ['BlockUniqueID', 'BlockNumber', 'SubBlockNumber']

blocks = gpd.read_file(product_path, layer = 'Blocks', include_fields = blocks_cols, ignore_geometry = True)
del blocks_cols

print('         input 5/7 read')

<h1 style="text-align:right;font-size:125%">
(פיצול לתהליכים יוצרים ותהליכים מעדכנים) נתוני שכבת גבולות תהליכי קדסטר

In [ ]:
cpb_cols = ['CPBUniqueID', 'ProcessName', 'ProcessType']

cpb_layer = gpd.read_file(product_path, layer = 'CadasterProcessBorders', include_fields = cpb_cols, ignore_geometry = True)


createprocess_layer = cpb_layer[cpb_layer['ProcessType'].isin([1,3,9])] #תצ"ר, פס"ד והסדר
del [cpb_cols, cpb_layer]


print('         input 6/7 read')

<h1 style="text-align:right;font-size:125%">
שכבת חלקות דו-ממדיות מודרנית

In [ ]:
Parcels2D_cols = ['ParcelNumber', 'BlockUniqueID']

Parcels2D_layer = gpd.read_file(product_path, layer = 'Parcels2D', include_fields = Parcels2D_cols, ignore_geometry = True)

del Parcels2D_cols

print('         input 7/7 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  15.b Cleaning data')

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני שכבת חלקות היסטוריות

<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם תכנית מלא לצורך מיזוג
<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גבולות תהליכי קדסטר יוצרים

In [ ]:
history_parcels[['TALAR_NUM','TALAR_YEAR']] = history_parcels[['TALAR_NUM','TALAR_YEAR']].fillna(0)
history_parcels[['TALAR_NUM','TALAR_YEAR']] = history_parcels[['TALAR_NUM','TALAR_YEAR']].astype(int)
history_parcels['ProcessName'] = history_parcels['TALAR_NUM'].astype(str) + '/' + history_parcels['TALAR_YEAR'].astype(str)
history_parcels.drop(columns =['TALAR_NUM','TALAR_YEAR'], inplace=True)

history_parcels = history_parcels.merge(right = createprocess_layer,
                                        on = 'ProcessName',
                                        how ='left')

history_parcels['CPBUniqueID'] = history_parcels['CPBUniqueID'].astype('Int64')

<h1 style="text-align:right;font-size:100%">
הסרת רשומות כפולות

In [ ]:
history_parcels.drop_duplicates(keep='first', inplace=True)

<h1 style="text-align:right;font-size:100%">
יצירת שדה של מספר גוש מלא לצורך מיזוג
<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גושי קדסטר מודרנית

In [ ]:
blocks['BlockName'] = blocks['BlockNumber'].astype(str) + '_' + blocks['SubBlockNumber'].astype(str)
history_parcels['BlockName'] = history_parcels['GUSH_NUM'].astype(str) + '_' + history_parcels['GUSH_SUFFIX'].astype(str)

history_parcels['BlockUniqueID'] = history_parcels.merge(right = blocks[['BlockUniqueID', 'BlockName']],
                                                         on = 'BlockName',
                                                         how = 'left')['BlockUniqueID']

history_parcels.drop(columns=['GUSH_NUM', 'GUSH_SUFFIX', 'BlockName', 'ProcessName'], inplace=True)
history_parcels['BlockUniqueID'] = history_parcels['BlockUniqueID'].astype('Int64')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות קיימים לשדות חדשים

In [ ]:
history_parcels.rename(columns = {'CPBUniqueID' : 'CPBUniqueCancelProcessID',
                                  'ProcessType' : 'CancelProcessType',
                                  'PARCEL_TYPE' : 'ParcelType',
                                  'PARCEL'      : 'ParcelNumber',
                                  'LEGAL_AREA'  : 'LegalArea'},
                       inplace = True)

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני טבלת חלקות מבוטלות

<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם תכנית מלא לצורך מיזוג
<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גבולות תהליכי קדסטר יוצרים
<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות קיימים לשדות חדשים

In [ ]:
cancel_parcel[['TALAR_NUMBER','TALAR_YEAR']] = cancel_parcel[['TALAR_NUMBER','TALAR_YEAR']].fillna(0)
cancel_parcel[['TALAR_NUMBER','TALAR_YEAR']] = cancel_parcel[['TALAR_NUMBER','TALAR_YEAR']].astype(int)
cancel_parcel['ProcessName'] = cancel_parcel['TALAR_NUMBER'].astype(str) + '/' + cancel_parcel['TALAR_YEAR'].astype(str)
cancel_parcel.drop(columns =['TALAR_NUMBER','TALAR_YEAR'], inplace = True)

cancel_parcel = cancel_parcel.merge(right = createprocess_layer,
                                    on = 'ProcessName',
                                    how = 'left')

cancel_parcel['CPBUniqueID'] = cancel_parcel['CPBUniqueID'].astype('Int64')

cancel_parcel.rename(columns = {'CPBUniqueID'  : 'CPBUniqueCancelProcessID',
                                'ProcessType'  : 'CancelProcessType',
                                'F_PARCEL_NUM' : 'ParcelNumber'},
                     inplace = True)

<h1 style="text-align:right;font-size:100%">
הסרת רשומות כפולות
<h1 style="text-align:right;font-size:100%">
הסרת מספרי חלקות לא תקינים

In [ ]:
cancel_parcel.drop_duplicates(keep = 'first', inplace = True)
cancel_parcel = cancel_parcel[~cancel_parcel['ParcelNumber'].isin([0, 9999])]

<h1 style="text-align:right;font-size:100%">
יצירת שדה של מספר גוש מלא לצורך מיזוג
<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גושי קדסטר מודרנית

In [ ]:
cancel_parcel[['F_GUSH_NUM', 'F_GUSH_SUFFIX']] = cancel_parcel[['F_GUSH_NUM', 'F_GUSH_SUFFIX']].fillna(0)
cancel_parcel[['F_GUSH_NUM', 'F_GUSH_SUFFIX']] = cancel_parcel[['F_GUSH_NUM', 'F_GUSH_SUFFIX']].astype(int)
cancel_parcel['BlockName'] = cancel_parcel['F_GUSH_NUM'].astype(str) + '_' + cancel_parcel['F_GUSH_SUFFIX'].astype(str)

cancel_parcel = cancel_parcel.merge(right = blocks[['BlockUniqueID', 'BlockName']],
                                                     on = 'BlockName',
                                                     how = 'left')

cancel_parcel['BlockUniqueID'] = cancel_parcel['BlockUniqueID'].astype('Int32')

cancel_parcel.drop(columns=['F_GUSH_NUM', 'F_GUSH_SUFFIX', 'BlockName', 'ProcessName'], inplace=True)

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני שכבת חלקות מבוטלות מרפליקציות יומיות

<h1 style="text-align:right;font-size:100%">
הסרת כפילויות

In [ ]:
replicated_parcels.drop_duplicates(keep = 'first', inplace = True)

<h1 style="text-align:right;font-size:100%">
יצירת שדה של מספר גוש מלא לצורך קבלת מספר מזהה לגוש

In [ ]:
replicated_parcels[['GUSH_NUM', 'GUSH_SUFFIX']] = replicated_parcels[['GUSH_NUM', 'GUSH_SUFFIX']].fillna(0)
replicated_parcels[['GUSH_NUM', 'GUSH_SUFFIX']] = replicated_parcels[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
replicated_parcels['BlockName'] = replicated_parcels['GUSH_NUM'].astype(str) + '_' + replicated_parcels['GUSH_SUFFIX'].astype(str)

replicated_parcels['BlockUniqueID'] = replicated_parcels.merge(right = blocks[['BlockUniqueID', 'BlockName']],
                                                               on = 'BlockName',
                                                               how = 'left')['BlockUniqueID']

replicated_parcels.drop(columns = ['GUSH_NUM', 'GUSH_SUFFIX', 'BlockName'], inplace=True)
replicated_parcels['BlockUniqueID'] = replicated_parcels['BlockUniqueID'].astype('Int64')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות קיימים לשדות חדשים

In [ ]:
replicated_parcels.rename(columns = {'PARCEL'    : 'ParcelNumber',
                                     'PARCEL_ID' : 'ParcelUniqueID'},
                          inplace = True)

<h1 style="text-align:right;font-size:125%;">
:איחוד נתונים 

In [ ]:
print('  15.c Data conversion')

<h1 style="text-align:right;font-size:100%">
קיבוץ רשומות לפי מספר חלקה ומספר מזהה של גוש
<h1 style="text-align:right;font-size:100%">
תיקון סוגי שדות ומילוי ערכים חסרים ב-0
<h1 style="text-align:right;font-size:100%">
הסרת רשומות בהן מספר החלקה הינו 0 
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש למספר מזהה של חלקה מבוטל

In [ ]:
CancelParcels2D =  history_parcels.append(cancel_parcel)\
                                  .append(replicated_parcels)

del [history_parcels, cancel_parcel, replicated_parcels]

In [ ]:
aggs = {'ParcelType' : 'max',
        'LegalArea'  : 'max',
        'geometry'   : 'first',
        'CancelProcessType' : 'max',
        'CPBUniqueCancelProcessID' : 'max'}

CancelParcels2D = CancelParcels2D.groupby(by = ['ParcelNumber', 'BlockUniqueID'],
                                          as_index = False,
                                          dropna   = False).agg(aggs)
del aggs

In [ ]:
cols_to_fill = ['ParcelType', 'BlockUniqueID','LegalArea', 'CPBUniqueCancelProcessID', 'CancelProcessType']

CancelParcels2D[cols_to_fill] = CancelParcels2D[cols_to_fill].fillna(0)

del cols_to_fill

In [ ]:
types_dict = {'ParcelNumber'      : 'Int64',
              'BlockUniqueID'     : 'Int64',
              'ParcelType'        : 'Int64',
              'LegalArea'         : float,
              'CPBUniqueCancelProcessID' : 'Int64',
              'CancelProcessType' : 'Int64'}

CancelParcels2D = CancelParcels2D.astype(types_dict)
del types_dict

<h1 style="text-align:right;font-size:100%">
הסרת חלקות שמספר הינו 0 או שמספר הגוש שלהן לא נמצא במיזוג

In [ ]:
CancelParcels2D = CancelParcels2D[CancelParcels2D['ParcelNumber'] != 0]
CancelParcels2D = CancelParcels2D[CancelParcels2D['BlockUniqueID'] != 0]

In [ ]:
CancelParcels2D['ParcelUniqueID'] = range(1, len(CancelParcels2D) + 1)
CancelParcels2D['ParcelUniqueID'] = CancelParcels2D['ParcelUniqueID'].astype(int)

<h1 style="text-align:right;font-size:100%">
הסרת חלקות מבוטלות שקיימות בשכבת חלקות רצף (עקב בעיות בשכבת רפליקציות)

In [ ]:
Parcels2D_layer['BlockUniqueID'] = Parcels2D_layer['BlockUniqueID'].fillna(0)

Parcels2D_layer['ParcelKey'] = Parcels2D_layer['ParcelNumber'].astype(str) + '-' + Parcels2D_layer['BlockUniqueID'].astype(str)
CancelParcels2D['ParcelKey'] = CancelParcels2D['ParcelNumber'].astype(str) + '-' + CancelParcels2D['BlockUniqueID'].astype(str)

In [ ]:
CancelParcels2D = CancelParcels2D[~CancelParcels2D['ParcelKey'].isin(Parcels2D_layer['ParcelKey'])]
CancelParcels2D.drop(columns = ['ParcelKey'], inplace=True)
del [Parcels2D_layer, blocks]

<h1 style="text-align:right;font-size:100%">
המרת שטח רשום מדונם למטר רבוע

In [ ]:
CancelParcels2D['LegalArea'] = CancelParcels2D['LegalArea'] * 1000

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  15.e Exporting results to product folder')

In [ ]:
CancelParcels2D_geom = CancelParcels2D[~CancelParcels2D['geometry'].isna()]
CancelParcels2D_geom = gpd.GeoDataFrame(CancelParcels2D_geom, geometry = 'geometry', crs = 2039)


CancelParcels2D_no_geom = CancelParcels2D[CancelParcels2D['geometry'].isna()]

In [ ]:
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\CancelParcels2D.geojson'
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\CancelParcels2D_no_geom.csv'


CancelParcels2D_no_geom.to_csv(no_geom_csv_dir, encoding = 'cp1255', index=False)

CancelParcels2D_geom.to_file(json_dir, driver = 'GeoJSON')


del [json_dir, no_geom_csv_dir, CancelParcels2D_geom, CancelParcels2D_no_geom, CancelParcels2D]

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\CancelParcels2D-Save as Feature Class.bat"]);